In [1]:
%load_ext autoreload
%autoreload 2
# test if commit works

In [2]:
import os
import sys
import pickle as pkl
from argparse import ArgumentParser
from copy import deepcopy
from os.path import join as oj
import numpy as np
import torch
from torch import nn
import torch.utils.data
from torch import optim
import pickle
from torch.utils.data import DataLoader, TensorDataset
sys.path.insert(0, "../code_fga")
from sklearn import datasets, metrics, model_selection, svm
from sklearn.metrics import RocCurveDisplay, roc_curve,roc_auc_score
import data_fns
import utils
import pandas as pd
import models
from matplotlib import pyplot as plt
cuda = torch.cuda.is_available()

if not cuda:
    sys.exit()
device = torch.device("cuda")

import torch.nn.utils.prune as prune
from sklearn import metrics
import configparser
config = configparser.ConfigParser()
colors = ['#5a05fa', '#320594', '#11ccb9', '#018786', '#cb3461', '#792039', '#f5660a', '#8e3d0b',]
# colors = colors[::2]+ colors[1::2]
config.read("../config.ini")

['../config.ini']

In [3]:
data_path = config["PATHS"]["data_path"]
model_path = config["PATHS"]["model_path"]

fig_path = config["PATHS"]["fig_path"]
PATH_IR_INDEX = oj(data_path,'ir_spectra_index.pkl')
PATH_IR_DATA = oj(data_path,'ir_spectra_data.pkl')


with open(PATH_IR_DATA, 'rb') as pickle_file:
    irdata = pickle.load(pickle_file)
    
with open(PATH_IR_INDEX, 'rb') as pickle_file:
    irindex = pickle.load(pickle_file)
filters = (('state', 'gas'), ('yunits', 'ABSORBANCE'), ('xunits', '1/CM'))
x, y = utils.fixed_domain_filter(irdata, irindex, filters)
x /= x.max(axis=1)[:, None]  # (batch_size, features)

x = x[:, None]
train_idxs, val_idxs, test_idxs = data_fns.get_split(len(x), seed=42)


In [5]:
fnames = sorted([oj(model_path, fname) for fname in os.listdir(model_path) if "pkl" in fname]) 
results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in (fnames)] 
results_all = pd.concat(results_list, axis=1).T.infer_objects()


In [6]:
results_all = results_all[results_all.exp_name == 'FinalModels']
# results_all = results_all[results_all.num_conv == 4]

In [7]:
train_dataset = TensorDataset(
    *[torch.Tensor(input) for input in [x[train_idxs], y[train_idxs]]]
) 
true_y_train = y[train_idxs]
pred_y_train = np.zeros_like(y[train_idxs])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)


In [8]:
def get_best_thresholds(model, train_loader, y_train):
    for batch_idx, (
        x_cur,
        y_cur,
    ) in enumerate(train_loader):
        pred_y_train[batch_idx*64:batch_idx*64+x_cur.shape[0]] = model.forward(x_cur.cuda()).detach().cpu().numpy()
    best_threshold = np.zeros((y.shape[1]))

    for i in range(17):
        precision, recall, thresholds = metrics.precision_recall_curve(true_y_train[:,i],  pred_y_train[:,i])

        f1_scores = 2*recall*precision/(recall+precision+ 1e-20)
        best_f1_score = np.nanmax(f1_scores)
        best_arg = np.argmax(f1_scores ==best_f1_score)
        my_threshhold = thresholds[best_arg]
        
        best_threshold[i] = my_threshhold
    return best_threshold
def get_auc(model, x_used, y_used):
    aucs = np.zeros((y_used.shape[1]))
    pred_used =model.forward(torch.Tensor(x_used).cuda()).detach().cpu().numpy()
    for i in range(len(aucs)):
        aucs[i] =roc_auc_score(y_used[:,i], pred_used[:,i])
    return aucs

def get_prec_recall_f1(model, x_used, y_used, thresholds):
    best_f1 = np.zeros((y_used.shape[1]))
    best_precision = np.zeros((y_used.shape[1]))
    best_recall = np.zeros((y_used.shape[1]))
    best_accuracy = np.zeros((y_used.shape[1]))
    pred_used = model.forward(torch.Tensor(x_used).cuda()).detach().cpu().numpy() > thresholds[None, :]


    for i in range(17):
        best_precision[i] = (pred_used[:,i] * y_used[:,i])[np.where(pred_used[:,i])].mean()
        best_recall[i] = (pred_used[:,i] * y_used[:,i])[np.where(y_used[:,i])].mean()
        best_f1[i] = 2*best_recall[i]*best_precision[i] /(best_recall[i]+best_precision[i])
        best_accuracy[i] = (pred_used[:,i] == y_used[:,i]).mean()
    return best_precision, best_recall, best_f1,best_accuracy
#         precision, recall, thresholds = metrics.precision_recall_curve(true_y_train[:,i],  pred_y_train[:,i])

def calc_molecular_f1(class_y, true_y):
    non_zero_idxs = np.where(true_y.sum(axis=1) !=0 )

    
    molecular_precision = ((class_y[non_zero_idxs] * true_y[non_zero_idxs]).sum(axis=1) / (class_y[non_zero_idxs]+1e-20).sum(axis=1))
    
    molecular_recall = ((class_y[non_zero_idxs] * true_y[non_zero_idxs]).sum(axis=1) / true_y[non_zero_idxs].sum(axis=1))
    
    molecular_f1 = 2* (molecular_precision*molecular_recall)/(molecular_precision+molecular_recall+1e-20)


#     print(molecular_f1.mean())
    return molecular_f1.mean()
def calc_mol_perfection(class_y, true_y):
    return np.all(class_y == true_y,axis=1).mean()
    
    


In [9]:
columns = ['Molecular Perfection','Molecular F-1'] 
all_df_mol =pd.DataFrame(columns =columns)
for model_idx in range(len(results_all)):
    # load model
    model = models.FGANet(num_input= x.shape[2],
                      num_in_channels=1, 
                      num_output = y.shape[1], 
      
                      conv_channels = results_all.iloc[model_idx].num_conv,
                      stride = 1).to(device)
    model.load_state_dict(torch.load(oj(model_path,results_all.iloc[model_idx].filename+".pt")))
    model = model.to(device)
    model.eval();
    # calculate best thresholds
    best_thresholds = get_best_thresholds(model, train_loader, y[train_idxs])
    pred_used = model.forward(torch.Tensor(x[test_idxs]).cuda()).detach().cpu().numpy() > best_thresholds[None, :]
    mol_perf = calc_mol_perfection(pred_used, y[test_idxs])
    mol_f1 = calc_molecular_f1(pred_used, y[test_idxs])
    append_df = pd.DataFrame({'Molecular F-1': mol_f1, 
                    'Molecular Perfection': mol_perf, },index = [0])
    all_df_mol =pd.concat([all_df_mol, append_df])
    

In [10]:
all_df_mol.mean()

Molecular Perfection    0.588852
Molecular F-1           0.895756
dtype: float64

In [11]:
all_df_mol.std()

Molecular Perfection    0.014633
Molecular F-1           0.003372
dtype: float64

In [12]:
num_classes=y.shape[1]

In [13]:
columns = ['Group','Proportion', 'F1', 'AUC', 'Accuracy', 'Precision', 'Recall'] 
all_df =pd.DataFrame(columns =columns)
performance_dict = {functional_group:{col:[] for col in columns[1:]} for functional_group in [x.capitalize() for x in irindex.columns[8:]]}
for model_idx in range(len(results_all)):
    # load model
    model = models.FGANet(num_input= x.shape[2],
                      num_in_channels=1, 
                      num_output = y.shape[1], 
                      conv_channels = results_all.iloc[model_idx].num_conv,
                      stride = 1).to(device)
    model.load_state_dict(torch.load(oj(model_path,results_all.iloc[model_idx].filename+".pt")))
    model = model.to(device).eval()
    # calculate best thresholds
    best_thresholds = get_best_thresholds(model, train_loader, y[train_idxs])
    # calculate AUC, for each group 
    best_precision, best_recall, best_f1,best_accuracy = get_prec_recall_f1(model, x[test_idxs], y[test_idxs], best_thresholds)    
    best_auc = get_auc(model, x[test_idxs], y[test_idxs])
    for i in range(num_classes):
        
        append_df = pd.DataFrame({'Group':irindex.columns[8+i].capitalize(),
                    'Proportion': y[:,i].mean(), 
                    'F1': best_f1[i], 
                    'AUC': best_auc[i], 
                    'Accuracy': best_accuracy[i], 
                    'Precision': best_precision[i], 
                    'Recall': best_recall[i]},index = [0])
        for j in columns[1:]:
            
            performance_dict[append_df['Group'][0]][j].append(append_df[j][0])
            
            
        
        
        all_df =pd.concat([all_df, append_df])
    
    # add to dataframa
    # calculate molecular perfection    

In [14]:
norm_table = all_df.max(axis=0)

In [15]:
perf_dic_val_std = {}
for key, val in performance_dict.items():
    perf_dic_val_std[key] ={}
    for perf_name, values in val.items():
        
        perf_dic_val_std[key][perf_name] = (np.asarray(values).mean(), np.asarray(values).std())
        
#         print(perf_indicatior)
    


In [16]:
fgs = list(all_df.groupby('Group').mean().index)
fgs_prop_idxs = np.asarray(all_df.groupby('Group').mean()['Proportion']).argsort()[::-1]
fgs = [fgs[k] for k in fgs_prop_idxs]

In [17]:
from matplotlib import colormaps
import numpy as np
cmap = colormaps['plasma']
xcmap = np.linspace(0, 1., 100)
rgb = cmap(xcmap)

cols =  ['Group','Proportion', 'F1', 'AUC', 'Accuracy', 'Precision', 'Recall']
def gen_latex_table_rows(d: dict, norm_table: pd.DataFrame, cols: list, fname: str):
    prec = 2
    header = '&'.join(cols) + ' \\\\ \n'
    with open(fname, "w") as f:
        f.write(header)
        for fg in fgs:
            row = [fg]
            for col in cols[1:]:
                val, std = d[fg][col]
                prec = 2
                
                # prec = get_precision(std)
                if not col in ('Precision','Proportion' ):
                    
                    err = f'({str(std).lstrip("0.")[prec]})'
                else:
                    err = ''
                # print(val/1.1*norm_table[col])
                rbg = np.array(cmap(val/(1.1*norm_table[col])))[:3]
                rgb = ','.join([f"{x:.2f}".lstrip('0').strip(' ') for x in rbg])
                
                if not col == 'Proportion':
                    row.append((f'\cellcolor[rgb]{{{rgb}}} $' + str(val)[:2+prec] + str(err) + '$').strip())
                else:
                    row.append(f'${str(val)[:2+prec]}$')
                
            f.write(' & '.join(row) + ' \\\\ \n')

gen_latex_table_rows(perf_dic_val_std, norm_table, cols, 'final.latex')


# Test untypical groups

In [18]:

cor_func = np.corrcoef(y[train_idxs].T)
cor_func = np.tril(np.corrcoef(y[train_idxs].T), k=-1)

num_groups = 5
# give me the two dimensional indice of the highest total max of this matrix
idx = np.unravel_index(np.tril(np.abs(cor_func)).reshape(-1).argsort(), cor_func.shape)
num_fgs = cor_func.shape[0]
idxs =(idx[0][-num_groups:], idx[1][-num_groups:])

for i in range(num_groups):
    print(irindex.columns[8+idxs[0][i]], irindex.columns[8+idxs[1][i]], cor_func[idxs[0][i],idxs[1][i]])
    print(idxs[0][i], idxs[1][i], cor_func[idxs[1][i],idxs[0][i]])

aromatics alkene -0.26561346678983844
7 2 0.0
alkyl halides methyl -0.3579722740363634
8 1 0.0
carboxylic acids alcohols 0.4553884115678942
12 4 0.0
aromatics alkane -0.4613867699572819
7 0 0.0
ether esters 0.6070406179243166
13 9 0.0


In [19]:
# function to return the f1 score for a particular functional group if the other group is present/ not present
def get_AUC_score(which_model, group_of_interest, related_group, other_group_is_present,best_thresholds):
    
    cur_idxs = np.where((y[test_idxs][:,related_group] == other_group_is_present))[0]
    pred_used =model.forward(torch.Tensor(x[test_idxs][cur_idxs]).cuda()).detach().cpu().numpy()[:,group_of_interest]

    if len(cur_idxs) == 0 or y[test_idxs][cur_idxs][:, group_of_interest].mean() == 0 or y[test_idxs][cur_idxs][:, group_of_interest].mean() == 1:
       return -1
        # print(y[test_idxs][cur_idxs][:, group_of_interest])
    fpr, tpr, thresholds = roc_curve(y[test_idxs][cur_idxs][:, group_of_interest], pred_used)
    # print(fpr)

    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc
# get_AUC_score(model, idxs[0,0], idxs[1,0], False,best_thresholds)


In [20]:

# make a data frame with the columns functional group name, correlation coeff, f1 score, other group present
idxs = np.asarray(idxs)
df = pd.DataFrame(columns = [ 'Corr coeff', 'Group 1', 'Group 2', 'AUC$_{no Group 2}$' ,'AUC$_{Group 2}$'])

for model_idx in range(len(results_all)):
    # load model
    model = models.FGANet(num_input= x.shape[2],
                      num_in_channels=1, 
                      num_output = y.shape[1], 
                      conv_channels = results_all.iloc[model_idx].num_conv,
                      stride = 1).to(device)
    model.load_state_dict(torch.load(oj(model_path,results_all.iloc[model_idx].filename+".pt")))
    model = model.to(device).eval()
    
    best_thresholds = get_best_thresholds(model, train_loader, y[train_idxs])
    for i in range(5):
        new_row = pd.DataFrame({'Corr coeff':cor_func[idxs[0][i],idxs[1][i]], 
                                'Group 1':irindex.columns[8+idxs[0][i]],
                                'Group 2':irindex.columns[8+idxs[1][i]],
                                'AUC$_{no Group 2}$' : get_AUC_score(model, idxs[0,i], idxs[1,i], False,best_thresholds),
                                'AUC$_{Group 2}$' : get_AUC_score(model, idxs[0,i], idxs[1,i], True,best_thresholds),
                                }, index = [0])
        
        df = pd.concat([df, new_row], ignore_index=True)
        new_row = pd.DataFrame({'Corr coeff':cor_func[idxs[0][i],idxs[1][i]], 
                                'Group 1':irindex.columns[8+idxs[1][i]],
                                'Group 2':irindex.columns[8+idxs[0][i]],
                                'AUC$_{no Group 2}$' : get_AUC_score(model, idxs[1,i], idxs[0,i], False,best_thresholds),
                                'AUC$_{Group 2}$' : get_AUC_score(model, idxs[1,i], idxs[0,i], True,best_thresholds),
                                }, index = [0])
        df = pd.concat([df, new_row], ignore_index=True)
        # break
    break
df = df.groupby(['Group 1', 'Group 2']).mean()
#sort by correlation coefficient
df = df.sort_values(by=['Corr coeff'], ascending=True)

In [21]:
df

,,Corr coeff,AUC$_{no Group 2}$,AUC$_{Group 2}$
Group 1,Group 2,,,
alkane,aromatics,-0.461387,0.951968,0.950379
aromatics,alkane,-0.461387,0.986920,0.995572
alkyl halides,methyl,-0.357972,0.900852,0.912542
methyl,alkyl halides,-0.357972,0.950225,0.977017
alkene,aromatics,-0.265613,0.969959,0.910782
aromatics,alkene,-0.265613,0.995744,0.993894
alcohols,carboxylic acids,0.455388,0.978628,-1.000000
carboxylic acids,alcohols,0.455388,-1.000000,0.988856
esters,ether,0.607041,-1.000000,0.970622


In [22]:
# to latex
latex_string = str(df.to_latex(index=True,float_format='%.2f')).replace('\cline{1-5}', '')
print(latex_string.replace("-1.00", "N/A"))
# sorted_df

\begin{tabular}{llrrr}
\toprule
 &  & Corr coeff & AUC$_{no Group 2}$ & AUC$_{Group 2}$ \\
Group 1 & Group 2 &  &  &  \\
\midrule
alkane & aromatics & -0.46 & 0.95 & 0.95 \\

aromatics & alkane & -0.46 & 0.99 & 1.00 \\

alkyl halides & methyl & -0.36 & 0.90 & 0.91 \\

methyl & alkyl halides & -0.36 & 0.95 & 0.98 \\

alkene & aromatics & -0.27 & 0.97 & 0.91 \\

aromatics & alkene & -0.27 & 1.00 & 0.99 \\

alcohols & carboxylic acids & 0.46 & 0.98 & N/A \\

carboxylic acids & alcohols & 0.46 & N/A & 0.99 \\

esters & ether & 0.61 & N/A & 0.97 \\

ether & esters & 0.61 & 0.98 & N/A \\

\bottomrule
\end{tabular}

